# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

ModuleNotFoundError: ignored

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [18]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

data = pd.read_csv('preprocessed_descriptions.csv')
descriptions = data['description'].dropna()

words = set()
for description in descriptions:
    tokens = word_tokenize(description)
    words.update(tokens)

print(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'smoothie-style', 'chest', '1980s', '4.1g', 'summer', 'jean-georges', 'dante', 'lamme', 'harbor', 'zaarian', 'cheez-its', 'shaped', '51349.', 'pages', 'remodeled', 'liberia', 'ones', 'feeling', '1-pot', 'marie', 'cast', 'emmantaler', 'jalapenos', 'dollar', 'flans', 'baba', 'geschnetzeltes', 'removal', 'soy/oil', 'shakelike', 'kadami', 'also.i', 'centuries', 'goodie', 'forefront', 'prized', 'wally', 'brooklyn', 'discribed', 'booklet.i', 'handeling', 'half-way', 'focaccia', 'marjoram', 'stools', 'masterfoods', 'etc.', 'dlife', 'helga', 'when', 'tori', 'flowerets', 'kettle', 'litte', 'enjoyed.time', 'gi', 'hellman', 'cheryl', 'aborio', 'ganache-', 'desired', 'garlic-thyme', 'sat.fat', 'tone', 'ballet', 'cattlemens', 'virtuous', 'brick', 'kirstie', 'mice', '1952', 'desided', 'bueno', 'spicing', 'feedback', 'germain', 'characteristic', 'adjusting', 'peasy', 'pyramid', 'rugelach', 'campus', 'romance', 'goldmine', 'anglo-indian', 'realistically', '250', 'mix-n-match', 'instructs', 'mojo', 'f

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [9]:
pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.5 MB/s eta 0:00:00


In [10]:
import random
from Levenshtein import distance as levenshtein_distance

random_word_pairs = [(random.choice(list(words)), random.choice(list(words))) for _ in range(5)]

for pair in random_word_pairs:
    dist = levenshtein_distance(pair[0], pair[1])
    print(f"Расстояние редактирования между '{pair[0]}' и '{pair[1]}': {dist}")

Расстояние редактирования между '6th-12th'06' и 'fluffy': 11
Расстояние редактирования между 'count' и 'knead': 5
Расстояние редактирования между 'everybody' и 'vegan': 7
Расстояние редактирования между 'office' и 'realized': 6
Расстояние редактирования между 'bit' и 'recommend': 9


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [11]:
def find_closest_words(word, words, k=5):
    distances = [(w, levenshtein_distance(word, w)) for w in words]
    distances.sort(key=lambda x: x[1])
    return [w[0] for w in distances[:k]]

word = "example"
closest_words = find_closest_words(word, words)
print(f"{word} ближайшие слова: {closest_words}")


example ближайшие слова: ['maple', 'simple', 'couple', 'email', 'excuse']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [12]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
nltk.download('wordnet')

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

word_data = []

for w in words:
    stemmed_word = stemmer.stem(w)
    normalized_word = lemmatizer.lemmatize(w)
    word_data.append({"word": w, "stemmed_word": stemmed_word, "normalized_word": normalized_word})

word_df = pd.DataFrame(word_data).set_index('word')
print(word_df)

[nltk_data] Downloading package wordnet to /root/nltk_data...


           stemmed_word normalized_word
word                                   
actual           actual          actual
cheddar         cheddar         cheddar
more               more            more
peas                pea             pea
couple            coupl          couple
...                 ...             ...
mind               mind            mind
cake               cake            cake
of                   of              of
loves              love            love
unbeatable       unbeat      unbeatable

[1953 rows x 2 columns]


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [20]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text, stop_words):
    tokens = word_tokenize(text)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return ' '.join(filtered_tokens)

filtered_descriptions = data['description'].dropna().apply(lambda x: remove_stopwords(x, stop_words))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
total_word_count = sum([len(word_tokenize(desc)) for desc in descriptions])
filtered_word_count = sum([len(word_tokenize(desc)) for desc in filtered_descriptions])
stopword_count = total_word_count - filtered_word_count
stopword_ratio = stopword_count / total_word_count

print(f"Доля стоп-слов: {stopword_ratio:.2f}")

Доля стоп-слов: 0.40


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

sampled_data = data.sample(5)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(sampled_data['description'])

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [22]:
from scipy.spatial.distance import cosine
import numpy as np

cosine_similarities = np.zeros((5, 5))
recipe_names = sampled_data['name'].values

for i in range(5):
    for j in range(5):
        cosine_similarities[i, j] = 1 - cosine(tfidf_matrix[i].toarray().flatten(), tfidf_matrix[j].toarray().flatten())

similarity_df = pd.DataFrame(cosine_similarities, columns=recipe_names, index=recipe_names)
print(similarity_df)

                                               ham and cheese breakfast strudels  \
ham and cheese breakfast strudels                                       1.000000   
smothered rabbit with tarragon sauce  chicken                           0.091840   
coconut lime shrimp skewers                                             0.243275   
lemon tarragon chicken with pan sauce                                   0.062167   
pork chops in creamy herbed brandy sauce                                0.270870   

                                               smothered rabbit with tarragon sauce  chicken  \
ham and cheese breakfast strudels                                                   0.091840   
smothered rabbit with tarragon sauce  chicken                                       1.000000   
coconut lime shrimp skewers                                                         0.096005   
lemon tarragon chicken with pan sauce                                               0.046083   
pork chops in c

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [24]:
most_similar_value = -1
most_similar_pair = None

for i in range(5):
    for j in range(5):
        if i != j and cosine_similarities[i, j] > most_similar_value:
            most_similar_value = cosine_similarities[i, j]
            most_similar_pair = (i, j)

print(f"Наиболее похожие рецепты: '{recipe_names[most_similar_pair[0]]}' и '{recipe_names[most_similar_pair[1]]}'. Косинусное расстояние: {most_similar_value:.2f}")

Наиболее похожие рецепты: 'ham and cheese breakfast strudels' и 'pork chops in creamy herbed brandy sauce'. Косинусное расстояние: 0.27
